<a href="https://colab.research.google.com/github/yasyamauchi/education/blob/main/Language_BME.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# N-gram  

"Ngram"は，キーワードの出現頻度を可視化する意味で用いられることが多い．  
[Google Ngram Viewer](https://books.google.com/ngrams) (Google Books)  
[NDL Ngram Viewer](https://lab.ndl.go.jp/ngramviewer/) (国立国会図書館)  

## N-gramをもとにして文章を生成してみる  
参考：[3. Pythonによる自然言語処理　1-1. 単語N-gram](https://qiita.com/y_itoh/items/82222af50bf1f80255eb) @y_itoh(yumi ito)  
  
この方法は純粋に訓練データ(テキスト)の単語の統計情報から文章を生成するものである．

In [1]:
with open('Neko.txt', mode='rt', encoding='utf-8') as f:
    read_text = f.read()
nekotxt = read_text

print(nekotxt)

吾輩は猫である。名前はまだ無い。どこで生れたかとんと見当がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。吾輩はここで始めて人間というものを見た。しかもあとで聞くとそれは書生という人間中で一番獰悪な種族であったそうだ。この書生というのは時々我々を捕えて煮て食うという話である。しかしその当時は何という考もなかったから別段恐しいとも思わなかった。ただ彼の掌に載せられてスーと持ち上げられた時何だかフワフワした感じがあったばかりである。掌の上で少し落ちついて書生の顔を見たのがいわゆる人間というものの見始であろう。この時妙なものだと思った感じが今でも残っている。第一毛をもって装飾されべきはずの顔がつるつるしてまるで薬缶だ。その後猫にもだいぶ逢ったがこんな片輪には一度も出会わした事がない。のみならず顔の真中があまりに突起している。そうしてその穴の中から時々ぷうぷうと煙を吹く。どうも咽せぽくて実に弱った。これが人間の飲む煙草というものである事はようやくこの頃知った。この書生の掌の裏でしばらくはよい心持に坐っておったが、しばらくすると非常な速力で運転し始めた。書生が動くのか自分だけが動くのか分らないが無暗に眼が廻る。胸が悪くなる。到底助からないと思っていると、どさりと音がして眼から火が出た。それまでは記憶しているがあとは何の事やらいくら考え出そうとしても分らない。ふと気が付いて見ると書生はいない。たくさんおった兄弟が一疋も見えぬ。肝心の母親さえ姿を隠してしまった。その上今までの所とは違って無暗に明るい。眼を明いていられぬくらいだ。はてな何でも容子がおかしいと、のそのそ這い出して見ると非常に痛い。吾輩は藁の上から急に笹原の中へ棄てられたのである。ようやくの思いで笹原を這い出すと向うに大きな池がある。吾輩は池の前に坐ってどうしたらよかろうと考えて見た。別にこれという分別も出ない。しばらくして泣いたら書生がまた迎に来てくれるかと考え付いた。ニャー、ニャーと試みにやって見たが誰も来ない。そのうち池の上をさらさらと風が渡って日が暮れかかる。腹が非常に減って来た。泣きたくても声が出ない。仕方がない、何でもよいから食物のある所まであるこうと決心をしてそろりそろりと池を左りに廻り始めた。どうも非常に苦しい。そこを我慢して無理やりに這って行くとようやくの事で何となく人

In [2]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  aptitude-common libcwidget4 libsigc++-2.0-0v5 libxapian30
Suggested packages:
  apt-xapian-index aptitude-doc-en | aptitude-doc debtags tasksel libcwidget-dev xapian-tools
The following NEW packages will be installed:
  aptitude aptitude-common libcwidget4 libsigc++-2.0-0v5 libxapian30
0 upgraded, 5 newly installed, 0 to remove and 24 not upgraded.
Need to get 3,838 kB of archives.
After this operation, 17.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 aptitude-common all 0.8.13-3ubuntu1 [1,719 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsigc++-2.0-0v5 amd64 2.10.4-2ubuntu3 [12.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libcwidget4 amd64 0.5.18-5build1 [306 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libxapian30 amd64 1

In [4]:
import MeCab
tagger = MeCab.Tagger("-Owakati")
nekotxt = tagger.parse(nekotxt)

# print(nekotxt)
nekotxt = nekotxt.split()
print(nekotxt)

['吾輩', 'は', '猫', 'で', 'ある', '。', '名前', 'は', 'まだ', '無い', '。', 'どこ', 'で', '生れ', 'た', 'か', 'とんと', '見当', 'が', 'つか', 'ぬ', '。', '何', 'でも', '薄暗い', 'じめじめ', 'し', 'た', '所', 'で', 'ニャーニャー', '泣い', 'て', 'いた事', 'だけ', 'は', '記憶', 'し', 'て', 'いる', '。', '吾輩', 'は', 'ここ', 'で', '始め', 'て', '人間', 'という', 'もの', 'を', '見', 'た', '。', 'しかも', 'あと', 'で', '聞く', 'と', 'それ', 'は', '書生', 'という', '人間', '中', 'で', '一番', '獰悪', 'な', '種族', 'で', 'あっ', 'た', 'そう', 'だ', '。', 'この', '書生', 'という', 'の', 'は', '時々', '我々', 'を', '捕え', 'て', '煮', 'て', '食う', 'という', '話', 'で', 'ある', '。', 'しかし', 'その', '当時', 'は', '何', 'という', '考', 'も', 'なかっ', 'た', 'から', '別段', '恐し', 'いとも', '思わ', 'なかっ', 'た', '。', 'ただ', '彼', 'の', '掌', 'に', '載せ', 'られ', 'て', 'スー', 'と', '持ち上げ', 'られ', 'た', '時', '何だか', 'フワフワ', 'し', 'た', '感じ', 'が', 'あっ', 'た', 'ばかり', 'で', 'ある', '。', '掌', 'の', '上', 'で', '少し', '落ちつい', 'て', '書生', 'の', '顔', 'を', '見', 'た', 'の', 'が', 'いわゆる', '人間', 'という', 'もの', 'の', '見', '始', 'で', 'あろ', 'う', '。', 'この', '時', '妙', 'な', 'もの', 'だ', 'と', '思っ', 'た', '感じ', 'が', '今', 'でも', '残

In [5]:
from collections import Counter
import numpy as np
from numpy.random import *

string = nekotxt

# 除外する文字記号
delimiter = ['「', '」', '…', '　']

# 2語のリスト
double = list(zip(string[:-1], string[1:]))
double = filter((lambda x: not((x[0] in delimiter) or (x[1] in delimiter))), double)

# 3語のリスト
triple = list(zip(string[:-2], string[1:-1], string[2:]))
triple = filter((lambda x: not((x[0] in delimiter) or (x[1] in delimiter) or (x[2] in delimiter))), triple)

# 要素数をカウントして辞書を生成
dic2 = Counter(double)
dic3 = Counter(triple)

In [6]:
for u,v in dic2.items():
    print(u, v)

ストリーミング出力は最後の 5000 行に切り捨てられました。
('あきれ', 'て') 1
('ながら', 'また') 1
('また', '進行') 1
('進行', 'する') 2
('それで', '置き') 1
('所', 'だけ') 1
('は', '出す') 1
('ただ', '出す') 1
('出す', 'だけ') 1
('なら', '人目') 1
('て', '眺める') 1
('眺める', 'くらい') 1
('は', 'やれん') 1
('やれん', '事') 1
('、', '眺め') 1
('眺め', 'た') 1
('。', '弾か') 1
('弾か', 'なけれ') 1
('ば', '役に立た') 1
('。', '弾け') 1
('弾け', 'ば') 1
('ば', '音') 1
('ちょうど', '木槿') 1
('木槿', '垣') 1
('重', '隔て') 1
('南', '隣り') 1
('沈澱', '組') 1
('の', '頭領') 1
('頭領', 'が') 1
('が', '下宿') 1
('から', '剣呑') 1
('に', '調子') 1
('合わせる', '。') 1
('こりゃ', '困る') 1
('。', '論') 1
('証拠', '音') 1
('小', '督') 1
('督', 'の') 1
('の', '局') 1
('局', 'も') 1
('全く', 'これ') 1
('で', 'しくじっ') 1
('が', 'ぬすみ') 1
('ぬすみ', '食') 1
('食', 'を') 1
('、', '贋札') 1
('を', '造る') 1
('造る', 'とか') 1
('まだ', '始末') 1
('、', '音曲') 1
('音曲', 'は') 1
('に', '隠し') 1
('隠し', 'ちゃ') 1
('ちゃ', '出来') 1
('出', 'なけれ') 1
('出来る', 'ん') 1
('。', '音') 1
('出', 'なけりゃ') 1
('なけりゃ', 'と') 1
('出', 'なく') 1
('隠し', '了') 1
('了', 'せ') 1
('が', '小石川') 1
('の', '御寺') 1
('御寺', 'で') 1
('で', '自炊') 1
('この', '藤

In [7]:
for u,v in dic3.items():
    print(u, v)

ストリーミング出力は最後の 5000 行に切り捨てられました。
('て', '台所', 'を') 1
('台所', 'を', 'かけ') 1
('を', 'かけ', '廻る') 2
('かけ', '廻る', '。') 2
('廻る', '。', '足音') 1
('。', '足音', 'は') 1
('足音', 'は', 'だんだん') 1
('は', 'だんだん', '近付い') 1
('だんだん', '近付い', 'て') 1
('近付い', 'て', 'くる') 1
('くる', '。', 'ああ') 1
('。', 'ああ', '残念') 1
('ああ', '残念', 'だ') 2
('残念', 'だ', 'が') 5
('だ', 'が', '天祐') 1
('が', '天祐', 'が') 1
('天祐', 'が', '少し') 1
('が', '少し', '足り') 1
('少し', '足り', 'ない') 1
('足り', 'ない', '。') 4
('ない', '。', 'とうとう') 2
('。', 'とうとう', '小') 1
('とうとう', '小', '供') 1
('小', '供', 'に') 3
('供', 'に', '見付け') 1
('に', '見付け', 'られ') 1
('見付け', 'られ', 'た') 1
('られ', 'た', '。') 6
('あら', '猫', 'が') 1
('猫', 'が', '御') 1
('が', '御', '雑煮') 1
('御', '雑煮', 'を') 3
('雑煮', 'を', '食べ') 2
('を', '食べ', 'て') 3
('食べ', 'て', '踊') 1
('て', '踊', 'を') 1
('踊', 'を', '踊っ') 1
('を', '踊っ', 'て') 1
('踊っ', 'て', 'いる') 1
('と', '大きな', '声') 10
('大きな', '声', 'を') 21
('声', 'を', 'する') 2
('する', '。', 'この') 2
('。', 'この', '声') 2
('この', '声', 'を') 1
('声', 'を', '第') 1
('を', '第', '一') 4
('一', 'に', '聞きつけ') 1
('に', '聞きつけ', '

KeyboardInterrupt: ignored

In [12]:
def nextword(words, dic):
    ## ➀先頭の単語wordsの要素数gramsを取得
    grams = len(words)

    ## ➁N-gram辞書dicから一致する要素を抽出
    # 2語の場合
    if grams == 2:
        matcheditems = np.array(list(filter(
            (lambda x: x[0][0] == words[1]), #1番目が合致
            dic.items())))
    # 3語の場合
    else:
        matcheditems = np.array(list(filter(
            (lambda x: x[0][0] == words[1]) and (lambda x: x[0][1] == words[2]), #1番目と2番目が合致
            dic.items())))

    ## ➂一致する語がない場合のエラーメッセージ
    if(len(matcheditems) == 0):
        print("No matched generator for", words[1])
        return ''

    ## ➃重み付き出現頻度リスト
    # matcheditemsから出現頻度を取得
    probs = [row[1] for row in matcheditems]
    # 0～1の疑似乱数を生成して出現頻度にかける
    weightlist = rand(len(matcheditems)) * probs

    ## ➄matcheditemsから重み付き出現頻度が最大の要素を取得
    if grams == 2:
        u = matcheditems[np.argmax(weightlist)][0][1]
    else:
        u = matcheditems[np.argmax(weightlist)][0][2]
    return u

# 先頭の単語wordsを入力
#words = ['', '短歌'] # 2-gram
words = ['', '明日', 'は'] # 3-gram

# 出力outputの先頭にwordsを埋め込む
output = words[1:]

# ｢次の語｣を取得
for i in range(100):
    # 2語の場合
    if len(words) == 2:
        newword = nextword(words, dic2)
    # 3語の場合
    else:
        newword = nextword(words, dic3)

    # 出力outputに次の語を追加
    output.append(newword)
    # 次の文字が終止符なら終了
    if newword in ['', '。', '？', '！']:
        break
    # 次のnextwordの準備
    words = output[-len(words):]
    print(words)

# 出力outputを表示
for u in output:
    print(u, end='')


['明日', 'は', '、']
['は', '、', 'その']
['、', 'その', '時']
['その', '時', '、']
['時', '、', '主人']
['、', '主人', 'は']
['主人', 'は', 'ない']
明日は、その時、主人はない。

<ipython-input-12-c501a1e47ed2>:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  matcheditems = np.array(list(filter(
